# Tarea Semanal 4

Yanina Corsaro

1- Código del ADC creado en la TS3:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sen import mi_funcion_sen
import scipy.signal as sig

def ADC(sr, Vf, bits):  
    
    #calculamos paso de cuantización
    q = Vf/(2**(bits-1))
    #cuantizamos la señal sr y normalizamos para que quede comprendida entre -Vf y +Vf
    sq= np.rint(sr/q)*q

    return sq

2- Creamos una senoidal contaminada con un nivel predeterminado de ruido.

In [ ]:
# parámetros de la función seno

N  = 1000 # muestras
fs = 1000 # Hz
p0 = np.pi/4# radianes
f0 = fs/N    # Hz
V = 2  # Volt

tt, xx= mi_funcion_sen( vmax=V , dc=0 , ff=f0 , ph=p0, nn=N , fs=fs )

